In [7]:
from collections import defaultdict
from aocutils.grid import gridneigh, neighbors


grid, neigh = gridneigh(
    "in.txt", diag=True, to_dict=True, parser=lambda x: [ch for ch in x])
grid = {k for k, v in grid.items() if v == "#"}

N = [(-1, 0), (-1, 1), (-1, -1)]
S = [(1, 0), (1, 1), (1, -1)]
W = [(0, -1), (1, -1), (-1, -1)]
E = [(0, 1), (-1, 1), (1, 1)]
orderpnt = 0
order = [N, S, W, E]

for step in range(1, 1000000):
    moved = False
    newgrid = defaultdict(set)
    nextround = set()

    for elf in grid:
        if any(n in grid for n in neighbors(elf, diag=True)):
            r, c = elf
            for i in range(4):
                o = order[(orderpnt + i) % 4]
                found = False
                for direc in o:
                    if (r + direc[0], c + direc[1]) in grid:
                        found = True
                        break
                if not found:
                    newgrid[(r + o[0][0], c + o[0][1])].add(elf)
                    break
            else:
                nextround.add(elf)
        else:
            nextround.add(elf)

    for newloc, elves in newgrid.items():
        if len(elves) > 1:
            for elf in elves:
                nextround.add(elf)
        else:
            nextround.add(tuple(newloc))
            moved = True
    orderpnt += 1
    grid = nextround

    if not moved:
        print("p2", step)
        break
    if step == 10:
        rows = [elf[0] for elf in grid]
        cols = [c for (r, c) in grid]
        print(
            "p1", (max(rows) - min(rows) + 1) * (max(cols) - min(cols) + 1) - len(grid)
        )

p1 4082
p2 1065
